In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch3d as p3d
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pytorch3d.structures import Meshes
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.io import load_obj, load_ply, save_ply
from pytorch3d.loss import chamfer_distance

In [2]:
### config

# 定义点云数据的输入维度和输出维度
input_dim = 3  # 每个点的特征维度
output_dim = 3  # 输出点云的特征维度（可以与输入维度相同）

device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 120
batch_size = 15

sxxxxes = os.listdir("../../data/all_results/")

In [3]:
# 创建一个简单的全连接神经网络类
class PointCloudFCNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PointCloudFCNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)  # 输入层到隐藏层1
        self.fc2 = nn.Linear(64, 128)  # 隐藏层1到隐藏层2
        self.fc3 = nn.Linear(128, 256)  # 隐藏层2到隐藏层3
        self.fc4 = nn.Linear(256, 128)  # 隐藏层3到隐藏层4
        self.fc5 = nn.Linear(128, output_dim)  # 隐藏层4到输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)  # 输出层不使用激活函数
        return x


# DataSet类
class TrainingSet(Dataset):
    def __init__(self, s_name_list, transform=None, sample_num=5000) -> None:
        self.input = [
            f"../../data/all_results/{sname}/0_sphere.obj" for sname in s_name_list
        ]
        self.target = [
            f"../../data/all_results/{sname}/{sname}.ply" for sname in s_name_list
        ]
        self.sample_num = sample_num
        self.transform = transform

    def __len__(self):
        return len(self.input)

    def __getitem__(self, index):
        #对input采样5000个点
        input_tensor_points, input_tensor_faces, _ = load_obj(self.input[index], load_textures=False)
        input_mesh = Meshes([input_tensor_points], [input_tensor_faces.verts_idx]) #注意这里有个.verts_idx
        input_tensor_points_sampled = sample_points_from_meshes(input_mesh,5000).squeeze(0)
        #对output采样5000个点
        target_tensor_points, target_tensor_faces = load_ply(self.target[index])
        target_mesh = Meshes([target_tensor_points],[target_tensor_faces])
        target_tensor_points_sampled = sample_points_from_meshes(target_mesh).squeeze(0)

        return input_tensor_points_sampled.to(device),target_tensor_points_sampled.to(device)

    def get_target(self, input):
        raise NotImplemented

In [4]:
#################################################################################
# pts,faces = load_ply(f"../../data/all_results/s0004_pulmonary_artery.nii.g_1/s0004_pulmonary_artery.nii.g_1.ply")
# mymesh = Meshes([pts],[faces])
# a = sample_points_from_meshes(mymesh,5)
# a = a.squeeze(0)
# a
# mymesh
#################################################################################

In [5]:
def plot_pointcloud(points, title=""):
    """Sample points uniformly from the surface of the mesh."""
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter3D(x, z, -y)
    ax.set_xlabel("x")
    ax.set_ylabel("z")
    ax.set_zlabel("y")
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

In [6]:
mydataset = TrainingSet(sxxxxes)
data_loader = DataLoader(mydataset, batch_size=batch_size, shuffle=False)


In [7]:
# 创建模型实例
model = PointCloudFCNet(input_dim, output_dim).to(device)
model

PointCloudFCNet(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=3, bias=True)
)

In [8]:
# 定义损失函数和优化器
criterion = chamfer_distance
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 使用Adam优化器

In [9]:
next(iter(data_loader))

[tensor([[[ 0.1609, -0.0926,  0.0879],
          [ 0.1922, -0.0395, -0.1166],
          [-0.0950,  0.1324,  0.1218],
          ...,
          [-0.1275, -0.2339,  0.0418],
          [ 0.1826, -0.1840, -0.0334],
          [-0.0225, -0.0075,  0.0850]],
 
         [[ 0.3944, -0.3060,  0.1218],
          [ 0.2779, -0.1932,  0.1360],
          [-0.0639,  0.0818,  0.3057],
          ...,
          [-0.3408, -0.2231,  0.2038],
          [-0.2917,  0.5264, -0.2750],
          [-0.1159, -0.2135,  0.1495]],
 
         [[ 0.2792, -0.0871, -0.1489],
          [-0.1138,  0.5535, -0.1634],
          [-0.3143,  0.3192, -0.0237],
          ...,
          [ 0.1172, -0.3036, -0.1346],
          [-0.0568,  0.4295,  0.1256],
          [ 0.6488, -0.2065, -0.0249]],
 
         ...,
 
         [[ 0.4073, -0.2485, -0.2905],
          [-0.2686,  0.6719, -0.0228],
          [-0.0322,  0.2010, -0.0364],
          ...,
          [ 0.4278, -0.2464, -0.0654],
          [-0.1487, -0.1017,  0.0570],
          [ 0.0996

In [10]:
# 训练模型

for epoch in range(num_epochs):
    for input_point_cloud,target_point_cloud in data_loader:
        optimizer.zero_grad()
        output_point_cloud = model(input_point_cloud)  # 前向传播
        loss,_ = criterion(output_point_cloud, target_point_cloud)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 优化模型参数

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    # if (epoch + 1) % 5 == 0:
    #     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # plot_pointcloud(output_point_cloud)# 绘制点云

Epoch [1/120], Loss: 0.0420
Epoch [2/120], Loss: 0.0210
Epoch [3/120], Loss: 0.0191
Epoch [4/120], Loss: 0.0178
Epoch [5/120], Loss: 0.0171
Epoch [6/120], Loss: 0.0160
Epoch [7/120], Loss: 0.0144
Epoch [8/120], Loss: 0.0127
Epoch [9/120], Loss: 0.0111
Epoch [10/120], Loss: 0.0100
Epoch [11/120], Loss: 0.0092
Epoch [12/120], Loss: 0.0083
Epoch [13/120], Loss: 0.0074
Epoch [14/120], Loss: 0.0066
Epoch [15/120], Loss: 0.0060
Epoch [16/120], Loss: 0.0057
Epoch [17/120], Loss: 0.0055
Epoch [18/120], Loss: 0.0053
Epoch [19/120], Loss: 0.0051
Epoch [20/120], Loss: 0.0050
Epoch [21/120], Loss: 0.0051
Epoch [22/120], Loss: 0.0050
Epoch [23/120], Loss: 0.0050
Epoch [24/120], Loss: 0.0049
Epoch [25/120], Loss: 0.0049
Epoch [26/120], Loss: 0.0049
Epoch [27/120], Loss: 0.0049
Epoch [28/120], Loss: 0.0048
Epoch [29/120], Loss: 0.0048
Epoch [30/120], Loss: 0.0048
Epoch [31/120], Loss: 0.0048
Epoch [32/120], Loss: 0.0047
Epoch [33/120], Loss: 0.0048
Epoch [34/120], Loss: 0.0047
Epoch [35/120], Loss: 0

In [12]:
# 使用训练好的模型生成新的点云数据
try_input_point_cloud,_,_ = load_obj("../../data/all_results/s1272_pulmonary_artery.nii.g_1/0_sphere.obj", load_textures=False)
new_point_cloud = model(try_input_point_cloud.to(device)) # TODO: 这里要改成用路径读一个点云
new_point_cloud
# 输出的new_point_cloud包含了经过神经网络处理后的新点云数据

tensor([[ 0.1073,  0.2392, -0.0715],
        [ 0.1028,  0.3097, -0.0674],
        [ 0.0845,  0.3496, -0.1211],
        ...,
        [ 0.8070, -0.1936, -0.2715],
        [ 0.7891, -0.1826, -0.2711],
        [ 0.7589, -0.2137, -0.2749]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [14]:
#看一眼新的输出长啥样
save_ply("./9-14-1106.ply",new_point_cloud)